In [4]:
class Node:
    def __init__(self, node, time, loose_edges_visited,pizzas, path):
        self.node = node
        self.time = time
        self.loose_edges_visited = loose_edges_visited
        self.pizzas = pizzas
        self.path = path

class World:
    def __init__(self, num_vertices, num_edges, edges, delivery_order, loose_edges, start_node, num_students, student_info):
        self.num_vertices = num_vertices
        self.num_edges = num_edges
        self.edges = edges
        self.delivery_order = delivery_order
        self.loose_edges = loose_edges
        self.start_node = start_node
        self.num_students = num_students
        self.student_info = student_info
        self.graph = self._build_graph()
        self.visited = set()

    @classmethod
    def parse(cls, filename):
        with open(filename, 'r') as f:
            num_vertices, num_edges = map(int, f.readline().strip().split())

            edges = []
            for _ in range(num_edges):
                edge = tuple(map(int, f.readline().strip().split()))
                edges.append(edge)

            num_loose_edges = int(f.readline().strip())
            loose_edges = {}
            for _ in range(num_loose_edges):
                edge, time = map(int, f.readline().strip().split())
                if edge not in loose_edges:
                    loose_edges[edge] = [time]
                else:
                    loose_edges[edge].append(time)

            start_node = int(f.readline().strip())
            num_students = int(f.readline().strip())

            student_info = []
            for _ in range(num_students):
                place, pizza_node = map(int, f.readline().strip().split())
                student_info.append((place, pizza_node))

            t = int(f.readline().strip())
            delivery_order = []
            for _ in range(t):
                b, a = map(int, f.readline().strip().split())
                delivery_order.append((b, a))

        return cls(num_vertices, num_edges, edges, delivery_order, loose_edges, start_node, num_students, student_info)

    def _build_graph(self):
        graph = [[] for _ in range(self.num_vertices + 1)]
        for edge in self.edges:
            graph[edge[0]].append(edge[1])
            graph[edge[1]].append(edge[0])
        return graph

    def _get_delivery_path(self):
        delivery_path = []
        for student in self.delivery_order:
            pizza_shop = self.student_info[student]
            delivery_path.append(pizza_shop)
            delivery_path.append(student)
        return delivery_path
    
    def bfs(self):
        # Initialize a queue with the starting node and no loose edges visited
        queue = [Node(self.start_node, 0, {}, [])]
    
        # Loop until the queue is empty
        while queue:
            # Pop the first node from the queue
            curr_node = queue.pop(0)
    
            # Check if we have delivered all pizzas, and return the time and loose edges visited
            if not self.delivery_order:
                return curr_node.time, curr_node.loose_edges_visited
    
            # Get the next delivery node
            next_delivery_node = self.delivery_order[0][1]
    
            # If we are at the next delivery node, deliver the pizza and remove it from the delivery order
            if curr_node.node == next_delivery_node:
                path = curr_node.path + [next_delivery_node]
                self.delivery_order.pop(0)
                if not self.delivery_order:
                    return curr_node.time, curr_node.loose_edges_visited
                next_delivery_node = self.delivery_order[0][1]
            else:
                path = curr_node.path
    
            # Check if we have visited this node with these loose edges before, and skip if we have
            if (curr_node.node, tuple(curr_node.loose_edges_visited.items())) in self.visited:
                continue
            
            # Mark this node as visited with these loose edges
            self.visited.add((curr_node.node, tuple(curr_node.loose_edges_visited.items())))
    
            # Loop over the neighbors of the current node
            for neighbor in self.graph[curr_node.node]:
                # If there is a loose edge between the current node and the neighbor, and we haven't visited it yet or
                # we visited it but its time has already passed, add the neighbor to the queue with the updated loose edges
                if (curr_node.node, neighbor) in self.loose_edges:
                    time = max(curr_node.time + 1, self.loose_edges[(curr_node.node, neighbor)][0])
                    if neighbor not in curr_node.loose_edges_visited or time < curr_node.loose_edges_visited[neighbor]:
                        loose_edges_visited = {**curr_node.loose_edges_visited, neighbor: time}
                        path_to_neighbor = path + [curr_node.node, neighbor]
                        queue.append(Node(neighbor, time, loose_edges_visited, path_to_neighbor))
                # If there is no loose edge between the current node and the neighbor, and we haven't visited it yet,
                # add the neighbor to the queue with the same loose edges as before
                else:
                    if neighbor not in curr_node.loose_edges_visited:
                        loose_edges_visited = curr_node.loose_edges_visited.copy()
                        path_to_neighbor = path + [curr_node.node, neighbor]
                        queue.append(Node(neighbor, curr_node.time + 1, loose_edges_visited, path_to_neighbor))
    
        # If we have not delivered all pizzas, return None
        return None, None


#    def ids(self):
#        start = self.start_node
#        max_depth = 0
#        while True:
#            result, _ = self._dls(start, max_depth)
#            if result is not None:
#                return result
#            max_depth += 1
#
#    def _dls(self, node, max_depth, time=0, loose_edges_visited={}):
#        if max_depth < 0:
#            return None, None
#        if not self.delivery_order:
#            return time, loose_edges_visited
#        next_delivery_node = self.delivery_order[0][1]
#        if node == next_delivery_node:
#            self.delivery_order.pop(0)
#            if not self.delivery_order:
#                return time, loose_edges_visited
#            next_delivery_node = self.delivery_order[0][1]
#        if (node, tuple(loose_edges_visited.items())) in self.visited:
#            return None, None
#        self.visited.add((node,  tuple(loose_edges_visited.items())))
#        for neighbor in self.graph[node]:
#            if (node, neighbor) in self.loose_edges:
#                if neighbor not in loose_edges_visited or time + 1 < loose_edges_visited[neighbor]:
#                    result, visited = self._dls(neighbor, max_depth - 1, time + 1, {**loose_edges_visited, neighbor: time + 1})
#                    if result is not None:
#                        return result, visited
#            else:
#                if neighbor not in loose_edges_visited:
#                    result, visited = self._dls(neighbor, max_depth - 1, time + 1, loose_edges_visited.copy())
#                    if result is not None:
#                        return result, visited
#        return None, None


In [2]:
world = World.parse('Test4.txt')
print("num_vertices: {}, num_edges:{}, edges: {}, loose_edges:{}, num_students:{}, student_info:{}, start_node:{}, delivery_order:{}".format(world.num_vertices,world.num_edges,world.edges,world.loose_edges,world.num_students,world.student_info,world.start_node,world.delivery_order))


num_vertices: 5, num_edges:4, edges: [(1, 2), (2, 3), (3, 4), (3, 5)], loose_edges:{3: [2]}, num_students:2, student_info:[(4, 3), (5, 2)], start_node:1, delivery_order:[(1, 2)]


This parse method reads the input file and returns an instance of the World class. It first reads the number of vertices and edges, and then the edges themselves. It then reads the number of loose edges and the details of each loose edge, storing the edge numbers and times in a dictionary. The start node, number of students, and details of each student are then read, followed by the delivery order. Finally, the parse method creates and returns a World object with all the parsed data.

In [45]:
from collections import deque

def bfs(num_vertices, edges, loose_edges, student_info, start_node, delivery_order):
    visited = set()
    queue = deque([(start_node, [], 0)])  # node, path, cost
    delivery_index = 0
    delivery_pos = {delivery_order[i][0]: i for i in range(len(delivery_order))}
    
    while queue:
        node, path, cost = queue.popleft()
        path = path + [node]
        visited.add(node)
        
        if node in delivery_pos:
            delivery = delivery_order[delivery_pos[node]]
            if delivery[1] == student_info[delivery_index][1]:
                delivery_index += 1
                if delivery_index == len(student_info):
                    return path, cost
            
        for neighbor in get_neighbors(node, edges, loose_edges):
            if neighbor not in visited:
                new_cost = cost + get_edge_cost(node, neighbor, loose_edges)
                queue.append((neighbor, path, new_cost))
                
    return [], float('inf')


def get_neighbors(node, edges, loose_edges):
    neighbors = set()
    for edge in edges:
        if edge[0] == node:
            neighbors.add(edge[1])
        elif edge[1] == node:
            neighbors.add(edge[0])
    if node in loose_edges:
        for neighbor in loose_edges[node]:
            neighbors.add(neighbor)
    return neighbors


def get_edge_cost(node1, node2, loose_edges):
    cost = 1
    if (node1, node2) in loose_edges or (node2, node1) in loose_edges:
        edge = (node1, node2) if (node1, node2) in loose_edges else (node2, node1)
        cost += loose_edges[edge[0]][edge[1] - 1]
    return cost

In [46]:
# Example usage with input
num_vertices = 5
num_edges = 4
edges = [(1, 2), (2, 3), (3, 4), (3, 5)]
loose_edges = {3: [2]}
num_students = 2
student_info = [(4, 3), (5, 2)]
start_node = 1
delivery_order = [(1, 2)]
path, cost = bfs(num_vertices, edges, loose_edges, student_info, start_node, delivery_order)
print("Path:", path)
print("Cost:", cost)


Path: []
Cost: inf


A*

In [31]:
from queue import PriorityQueue

def manhattan_distance(n1, n2):
    if isinstance(n1, int):
        x1, y1 = divmod(n1 - 1, num_cols)
        n1 = (x1, y1)
    if isinstance(n2, int):
        x2, y2 = divmod(n2 - 1, num_cols)
        n2 = (x2, y2)
    return abs(n1[0] - n2[0]) + abs(n1[1] - n2[1])

def astar_consistent(num_vertices, edges, loose_edges, student_info, start_node, delivery_order):
    # Create the graph
    graph = {}
    for node in range(1, num_vertices + 1):
        graph[node] = []
    for edge in edges:
        graph[edge[0]].append(edge[1])
        graph[edge[1]].append(edge[0])

    # Initialize the search with the start node
    priority_queue = PriorityQueue()
    priority_queue.put((manhattan_distance(start_node, delivery_order[0][0]), start_node, [], 0, 0))
    visited = set()
    while not priority_queue.empty():
        # Get the next node to explore
        _, current_node, path, path_cost, delivery_index = priority_queue.get()
        path = path + [current_node]

        # Check if the current node is the delivery location
        if current_node == delivery_order[delivery_index][0]:
            delivery_index += 1
            # Check if all deliveries have been made
            if delivery_index == len(delivery_order):
                return path, path_cost

        # Add neighboring nodes to the queue
        for neighbor in graph[current_node]:
            if (current_node, neighbor) in loose_edges:
                edge_cost = loose_edges[(current_node, neighbor)]
            else:
                edge_cost = 1
            new_cost = path_cost + edge_cost
            if neighbor not in visited:
                priority_queue.put((new_cost + manhattan_distance(neighbor, delivery_order[delivery_index][0]), neighbor, path, new_cost, delivery_index))
                visited.add(neighbor)
    return [], float('inf')


In [34]:
num_vertices = 5
num_edges = 4
edges = [(1, 2), (2, 3), (3, 4), (3, 5)]
loose_edges = {3: [2]}
num_students = 2
student_info = [(4, 3), (5, 2)]
start_node = 1
delivery_order = [(1, 2)]

# Call the astar_consistent function
path, cost = astar_consistent(num_vertices, edges, loose_edges, student_info, start_node, delivery_order)

# Print the results
print("A* Path:", path)
print("A* Cost:", cost)


A* Path: [1]
A* Cost: 0
